In [ ]:
!conda install '../input/covidnew/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '../input/covidnew/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '../input/covidnew/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '../input/covidnew/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '../input/covidnew/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '../input/covidnew/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
from torch import Tensor
from torch.utils.data import Dataset
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
import numpy as np
import torch
from torch import nn
from torch import optim
from numpy import asarray
import os

from PIL import Image
import pandas as pd

In [ ]:

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import numpy as np

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [ ]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=torch.load('../input/covidnew/model.pth')
model.eval()
col=['id','PredictionString']
test_transforms = transforms.Compose([transforms.ToTensor(),
                                     ])
prediction=pd.DataFrame(columns = col) 
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/test'):
    for filename in filenames:
        img_path= os.path.join(dirname, filename)
        id_=img_path.split('/')[5]+"_study"
        image=read_xray(img_path)
        image=resize(image, 416)
        image=asarray(image)
        image = torch.from_numpy(image).long()
        image=np.stack((image,)*3, axis=0)
        image_tensor = test_transforms(image).float()
        image_tensor = image_tensor.unsqueeze_(0)
        #inputs = Variable(image_tensor)
        inputs = image_tensor.to(device)
        inputs=torch.transpose(inputs, 1, 2)
        output = model(inputs)
        predProb = output.detach().cpu().numpy()[0]
        PredictionString=f'negative {format(predProb[0],".8f")} 0 0 1 1 typical {format(predProb[1],".5f")} 0 0 1 1 indeterminate {format(predProb[2],".5f")} 0 0 1 1 atypical {format(predProb[3],".5f")} 0 0 1 1' 
        #print(PredictionString)
        prediction=prediction.append({"id":id_,"PredictionString":PredictionString},ignore_index=True)

In [ ]:
def CreateSub(test):
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    for i in range(len(sub_df)):
        #print(test[test['id']==sub_df.loc[i,'id']])
        #negative, typical, indeterminate, atypical = str(tst_preds[i][0]),str(tst_preds[i][1]),str(tst_preds[i][2]),str(tst_preds[i][3]),
        try:
            sub_df.loc[i,'PredictionString'] =test[test['id']==sub_df.loc[i,'id']].iloc[0,1]# test[test['id']==sub_df.loc[i,'id']][0][1]#f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
        except:
            continue
    return sub_df

In [ ]:
sumfile = CreateSub(prediction)
sumfile.to_csv('./submission.csv',index=False)